### Библиотеки

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gmean

import statsmodels.api as sm
from scipy.stats import t, f, boxcox, skew, kurtosis
from statsmodels.stats.diagnostic import linear_reset, het_white


import warnings
warnings.filterwarnings('ignore')

### Обработка

In [50]:
data = pd.read_csv('data_after_processing.csv', encoding='utf-8',)

data = data.drop(['title'], axis=1)
data = data.drop(['author_Другой', 'publisher_Другой', 'publication_year_Другой',
                  'cover_type_Мягкий заламинированный картон', 'reading_age_6+'], axis=1)

### Логарифм цены

In [51]:
data['log_price'] = np.log(data['price'])

In [52]:
data.columns

Index(['price', 'avg_rating', 'cnt_reviews', 'pages_cnt', 'tirage', 'weight',
       'author_Джейн Остен', 'author_Джек Лондон', 'author_Джордж Оруэлл',
       'author_Лев Толстой', 'author_Луиза Мэй Олкотт',
       'author_Михаил Булгаков', 'author_Николай Гоголь',
       'author_Федор Достоевский', 'author_Эрих Ремарк', 'publisher_АСТ',
       'publisher_Азбука', 'publisher_Иностранка',
       'publisher_Манн, Иванов и Фербер', 'publisher_Эксмо',
       'publication_year_2021', 'publication_year_2022',
       'publication_year_2023', 'publication_year_2024',
       'publication_year_2025', 'cover_type_Мягкий переплёт',
       'cover_type_Твёрдый переплёт', 'reading_age_0+', 'reading_age_12+',
       'reading_age_16+', 'reading_age_18+', 'thickness', 'width', 'length',
       'volume', 'log_price'],
      dtype='object')

### Регрессоры, которые можно логарифмировать и нет

In [59]:
unconditional_cols = [
    'author_Джейн Остен', 'author_Джек Лондон', 'author_Джордж Оруэлл',
       'author_Лев Толстой', 'author_Луиза Мэй Олкотт',
       'author_Михаил Булгаков', 'author_Николай Гоголь',
       'author_Федор Достоевский', 'author_Эрих Ремарк', 'publisher_АСТ',
       'publisher_Азбука', 'publisher_Иностранка',
       'publisher_Манн, Иванов и Фербер', 'publisher_Эксмо',
       'publication_year_2021', 'publication_year_2022',
       'publication_year_2023', 'publication_year_2024',
       'publication_year_2025', 'cover_type_Мягкий переплёт',
       'cover_type_Твёрдый переплёт', 'reading_age_0+', 'reading_age_12+',
       'reading_age_16+', 'reading_age_18+',
]

cols_to_try_log = [
    'avg_rating', 'cnt_reviews', 'pages_cnt', 'tirage', 
    'weight', 'thickness', 'width', 'length', 'volume'
]

### Подбор функциональной формы для $\log{price}$

In [70]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from itertools import product
from tqdm import tqdm  # для прогресс-бара (опционально)


def generate_all_combinations():
    """Генерирует все комбинации логарифмирования для cols_to_try_log"""
    return product([False, True], repeat=len(cols_to_try_log))

def prepare_X(combination, df):
    """Создает матрицу X для текущей комбинации"""
    X_temp = df[unconditional_cols].copy()
    
    for i, col in enumerate(cols_to_try_log):
        if combination[i]:
            # Проверка на положительные значения
            if (df[col] <= 0).any():
                # Добавляем константу, чтобы избежать log(0)
                X_temp[f'log_{col}'] = np.log(df[col] + 1e-6)
            else:
                X_temp[f'log_{col}'] = np.log(df[col])
        else:
            X_temp[col] = df[col]
    
    return sm.add_constant(X_temp)

results = []

# Генерируем все комбинации (2^N вариантов)
total_combinations = 2 ** len(cols_to_try_log)

for combination in tqdm(generate_all_combinations(), total=total_combinations):
    try:
        # Подготавливаем данные
        X_curr = prepare_X(combination, data)
        
        # # Удаляем возможные NaN
        # X_curr_clean = X_curr.dropna(axis=1, how='any')
        # y_clean = y.reindex_like(X_curr_clean)
        
        # Строим модель
        model = sm.OLS(data['log_price'], X_curr).fit()
        
        # Сохраняем результаты
        results.append({
            'combination': combination,
            'aic': model.aic,
            'bic': model.bic,
            'adj_r2': model.rsquared_adj,
            'model': model
        })
    
    except Exception as e:
        print(f"Error in combination {combination}: {str(e)}")
        continue

# Создаем DataFrame с результатами
df_results = pd.DataFrame(results)

# Добавляем расшифровку комбинаций
df_results['log_columns'] = df_results['combination'].apply(
    lambda x: [cols_to_try_log[i] for i, log in enumerate(x) if log]
)

# Находим лучшие модели по разным критериям
best_aic = df_results.loc[df_results['aic'].idxmin()]
best_bic = df_results.loc[df_results['bic'].idxmin()]
best_adj_r2 = df_results.loc[df_results['adj_r2'].idxmax()]

print("Лучшая модель по AIC:")
print(f"Логарифмированные переменные: {best_aic['log_columns']}")
print(f"AIC: {best_aic['aic']:.2f}\n")

print("Лучшая модель по BIC:")
print(f"Логарифмированные переменные: {best_bic['log_columns']}")
print(f"BIC: {best_bic['bic']:.2f}\n")

print("Лучшая модель по Adj.R²:")
print(f"Логарифмированные переменные: {best_adj_r2['log_columns']}")
print(f"Adj.R²: {best_adj_r2['adj_r2']:.4f}")


100%|██████████| 512/512 [00:06<00:00, 84.27it/s]

Лучшая модель по AIC:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'weight', 'thickness']
AIC: -394.16

Лучшая модель по BIC:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'weight', 'thickness']
BIC: -181.07

Лучшая модель по Adj.R²:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'weight', 'thickness']
Adj.R²: 0.8077


### Подбор функциональной формы для $price$

In [61]:
results = []

# Генерируем все комбинации (2^N вариантов)
total_combinations = 2 ** len(cols_to_try_log)

for combination in tqdm(generate_all_combinations(), total=total_combinations):
    try:
        # Подготавливаем данные
        X_curr = prepare_X(combination, data)
        
        # # Удаляем возможные NaN
        # X_curr_clean = X_curr.dropna(axis=1, how='any')
        # y_clean = y.reindex_like(X_curr_clean)
        
        # Строим модель
        model = sm.OLS(data['price'], X_curr).fit()
        
        # Сохраняем результаты
        results.append({
            'combination': combination,
            'aic': model.aic,
            'bic': model.bic,
            'adj_r2': model.rsquared_adj,
            'model': model
        })
    
    except Exception as e:
        print(f"Error in combination {combination}: {str(e)}")
        continue

# Создаем DataFrame с результатами
df_results = pd.DataFrame(results)

# Добавляем расшифровку комбинаций
df_results['log_columns'] = df_results['combination'].apply(
    lambda x: [cols_to_try_log[i] for i, log in enumerate(x) if log]
)

# Находим лучшие модели по разным критериям
best_aic = df_results.loc[df_results['aic'].idxmin()]
best_bic = df_results.loc[df_results['bic'].idxmin()]
best_adj_r2 = df_results.loc[df_results['adj_r2'].idxmax()]

print("Лучшая модель по AIC:")
print(f"Логарифмированные переменные: {best_aic['log_columns']}")
print(f"AIC: {best_aic['aic']:.2f}\n")

print("Лучшая модель по BIC:")
print(f"Логарифмированные переменные: {best_bic['log_columns']}")
print(f"BIC: {best_bic['bic']:.2f}\n")

print("Лучшая модель по Adj.R²:")
print(f"Логарифмированные переменные: {best_adj_r2['log_columns']}")
print(f"Adj.R²: {best_adj_r2['adj_r2']:.4f}")


100%|██████████| 512/512 [00:06<00:00, 75.19it/s] 


Лучшая модель по AIC:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'width', 'volume']
AIC: 41424.55

Лучшая модель по BIC:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'width', 'volume']
BIC: 41637.64

Лучшая модель по Adj.R²:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'width', 'volume']
Adj.R²: 0.8541


### Вывод

Нужно логарифмировать ['pages_cnt', 'tirage', 'weight', 'thickness'] для модели с логарифмом цены

Нужно логарифмировать ['pages_cnt', 'tirage', 'width', 'thickness'] для модели с линейной ценой

### Итоговая модель с ln Y

In [72]:
cols_to_log = ['pages_cnt', 'tirage', 'weight', 'thickness']
cols_not_to_log = [col for col in cols_to_try_log if col not in cols_to_log] + unconditional_cols

X_log = np.log(data.loc[:, cols_to_log])
X_not_to_log = data.loc[:, cols_not_to_log]

X_log_model = sm.add_constant(pd.concat((X_log, X_not_to_log), axis=1))

log_model = sm.OLS(data['log_price'], X_log_model).fit()
log_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     403.0
Date:                Mon, 05 May 2025   Prob (F-statistic):               0.00
Time:                        00:06:53   Log-Likelihood:                 232.08
No. Observations:                3256   AIC:                            -394.2
Df Residuals:                    3221   BIC:                            -181.1
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               5.1408      0.285     18.012      0.000       4.581       5.700
pages_cnt                          -0.4177      0.022    -18.714      0.000      -0.461      -0.374
tirage                             -0.1707      0.010    -17.330      0.000      -0.190      -0.151
weight                              1.0142      0.022     46.985      0.000       0.972       1.057
thickness                          -0.4881      0.035    -14.044      0.000      -0.556      -0.420
avg_rating                          0.0484      0.012      4.132      0.000       0.025       0.071
cnt_reviews                         0.0003   3.46e-05      9.469      0.000       0.000       0.000
width                              -0.0394      0.003    -13.469      0.000      -0.045      -0.034
length                             -0.0164      0.004     -3.927      0.000      -0.025      -0.008
volume                              0.0006   3.63e-05     16.473      0.000       0.001       0.001
author_Джейн Остен                  0.0172      0.028      0.614      0.539      -0.038       0.072
author_Джек Лондон                 -0.0631      0.027     -2.300      0.022      -0.117      -0.009
author_Джордж Оруэлл                0.0110      0.029      0.381      0.704      -0.046       0.068
author_Лев Толстой                 -0.1110      0.031     -3.611      0.000      -0.171      -0.051
author_Луиза Мэй Олкотт            -0.0269      0.032     -0.833      0.405      -0.090       0.036
author_Михаил Булгаков              0.0758      0.027      2.856      0.004       0.024       0.128
author_Николай Гоголь              -0.0741      0.033     -2.266      0.024      -0.138      -0.010
author_Федор Достоевский           -0.0818      0.021     -3.914      0.000      -0.123      -0.041
author_Эрих Ремарк                  0.0881      0.029      3.027      0.002       0.031       0.145
publisher_АСТ                      -0.0882      0.030     -2.949      0.003      -0.147      -0.030
publisher_Азбука                   -0.0830      0.031     -2.676      0.007      -0.144      -0.022
publisher_Иностранка                0.0033      0.035      0.094      0.925      -0.065       0.072
publisher_Манн, Иванов и Фербер     0.0183      0.040      0.457      0.648      -0.060       0.097
publisher_Эксмо                    -0.1106      0.030     -3.654      0.000      -0.170      -0.051
publication_year_2021              -0.0390      0.045     -0.870      0.384      -0.127       0.049
publication_year_2022              -0.0056      0.038     -0.149      0.882      -0.079       0.068
publication_year_2023               0.0129      0.036      0.359      0.720      -0.058       0.084
publication_year_2024               0.0355      0.035      1.001      0.317      -0

### Итоговая модель с Y

In [74]:
cols_to_log = ['pages_cnt', 'tirage', 'width', 'thickness']
cols_not_to_log = [col for col in cols_to_try_log if col not in cols_to_log] + unconditional_cols

X_log = np.log(data.loc[:, cols_to_log])
X_not_to_log = data.loc[:, cols_not_to_log]

X_lin_model = sm.add_constant(pd.concat((X_log, X_not_to_log), axis=1))

lin_model = sm.OLS(data['price'], X_lin_model).fit()
lin_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                     543.5
Date:                Mon, 05 May 2025   Prob (F-statistic):               0.00
Time:                        00:07:13   Log-Likelihood:                -20723.
No. Observations:                3256   AIC:                         4.152e+04
Df Residuals:                    3221   BIC:                         4.173e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                            1678.5532    184.563      9.095      0.000    1316.680    2040.427
pages_cnt                        -197.6533     12.372    -15.976      0.000    -221.910    -173.396
tirage                            -88.2075      6.132    -14.384      0.000    -100.231     -76.184
width                             131.0668     23.696      5.531      0.000      84.605     177.528
thickness                         -58.6737     18.286     -3.209      0.001     -94.526     -22.821
avg_rating                         25.0838      7.281      3.445      0.001      10.809      39.359
cnt_reviews                         0.1448      0.022      6.716      0.000       0.103       0.187
weight                              1.5686      0.024     66.420      0.000       1.522       1.615
length                              6.1313      2.502      2.451      0.014       1.226      11.037
volume                             -0.0867      0.017     -5.250      0.000      -0.119      -0.054
author_Джейн Остен                  6.8611     17.495      0.392      0.695     -27.441      41.163
author_Джек Лондон                -13.9086     17.116     -0.813      0.417     -47.468      19.651
author_Джордж Оруэлл               20.7066     18.076      1.146      0.252     -14.735      56.148
author_Лев Толстой               -108.1757     19.148     -5.649      0.000    -145.719     -70.632
author_Луиза Мэй Олкотт           -10.7900     20.116     -0.536      0.592     -50.231      28.651
author_Михаил Булгаков             78.3644     16.548      4.736      0.000      45.919     110.810
author_Николай Гоголь             -40.3683     20.405     -1.978      0.048     -80.377      -0.360
author_Федор Достоевский          -43.4575     13.033     -3.334      0.001     -69.011     -17.904
author_Эрих Ремарк                 46.9596     18.146      2.588      0.010      11.381      82.539
publisher_АСТ                     -88.4881     18.613     -4.754      0.000    -124.984     -51.993
publisher_Азбука                 -117.7959     19.332     -6.093      0.000    -155.701     -79.891
publisher_Иностранка             -122.7748     22.013     -5.577      0.000    -165.935     -79.614
publisher_Манн, Иванов и Фербер   -72.6133     25.010     -2.903      0.004    -121.650     -23.576
publisher_Эксмо                  -101.1283     18.882     -5.356      0.000    -138.150     -64.107
publication_year_2021               5.4664     27.958      0.196      0.845     -49.351      60.284
publication_year_2022              19.6107     23.356      0.840      0.401     -26.184      65.405
publication_year_2023              28.4456     22.476      1.266      0.206     -15.623      72.514
publication_year_2024              36.8428     22.077      1.669      0.095      -6